### 0. Imports

In [108]:
# transformación de datos
import pandas as pd

# work with environment variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve database credentials
USERNAME = os.getenv("DATABASE_USERNAME")
PASSWORD = os.getenv("DATABASE_PASSWORD")

# append parent folder to path
import sys
sys.path.append("..")

# import data load support functions
import src.data_load_support as dls

# import database connection support functions
import src.database_connection_support as dcs

# 1. Introducción - Diseño de BDD y creación de tablas

Este notebook detalla la limpieza de los datos previa a la subida en base de datos.

# 2. Importacion

In [109]:
detalles_artistas = pd.read_csv("../data/cleaned/detalles_artistas.csv",index_col=0)
detalles_artistas.head(1)

,actr,año_nac,conocido_por,que_hace,premios
0,A. Gunaseelan,NaN,'Mandela',"Casting Department, Actress, Casting Director",1 nomination


In [110]:
detalles_peliculas = pd.read_csv("../data/cleaned/detalles_peliculas.csv", index_col=0)
detalles_peliculas.head(1)

,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,NaN,1h 35min,tt0320656


In [111]:
pelicula_artista = pd.read_csv("../data/cleaned/pelicula_artista.csv",index_col=0)
pelicula_artista.head(1)

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung


In [112]:
peliculas = pd.read_csv("../data/cleaned/peliculas.csv",index_col=0)
peliculas.head(1)

,Tipo,Titulo,Año,Mes,Id,generos
0,Movie,Las Travesuras de Super Chido,1990.0,10.0,tt0320656,Action


# 3. Esquema base de datos

peliculas
- id_pelicula PK
- tipo VARCHAR
- genero VARCHAR
- titulo VARCHAR
- anio INT
- mes INT
- cali_imdb FLOAT
- direc_imdb VARCHAR
- guion_imdb VARCHAR
- argum_imdb VARCHAR
- durac_imdb VARCHAR


peliculas_artistas
- id SERIAL PK
- id_pelicula INT FK
- actriz_actor VARCHAR FK

artistas
- actriz_actor FK
- anio_nac INT 
- que_hace VARCHAR
- premios VARCHAR

## 3.1 Creación de la base de datos

In [113]:
database_credentials = {"username":USERNAME,
                        "password":PASSWORD,
                        }

In [114]:
dls.create_db("prueba_tecnica", credentials_dict=database_credentials)

Database already existant.


## 3.2 Creacion de tablas

In [124]:
dropear_tablas = "DROP TABLE IF EXISTS peliculas, artistas, peliculas_artistas, peliculas_artista CASCADE;"

crear_peliculas = """
CREATE TABLE peliculas (
    id_pelicula VARCHAR(30) PRIMARY KEY,
    titulo VARCHAR (100),
    tipo VARCHAR(15),
    genero VARCHAR(20),
    anio NUMERIC, -- deberia ser INT, arreglo necesario para la subida rápida
    mes NUMERIC,-- deberia ser INT, arreglo necesario para la subida rápida
    cali_imdb NUMERIC,
    direc_imdb VARCHAR(255),
    guion_imdb VARCHAR(255),
    argum_imdb TEXT,
    durac_imdb VARCHAR(10)
);
"""

crear_artistas = """
CREATE TABLE artistas(
    actriz_actor VARCHAR(50) PRIMARY KEY,
    anio_nac NUMERIC,  -- deberia ser INT, arreglo necesario para la subida rápida
    que_hace VARCHAR(255),
    premios VARCHAR(255)
);
"""

crear_peliculas_artistas = """
CREATE TABLE peliculas_artistas (
    id SERIAL PRIMARY KEY,
    id_pelicula VARCHAR(30) REFERENCES peliculas(id_pelicula),
    actriz_actor VARCHAR(50) REFERENCES artistas(actriz_actor)
);
"""


Definir la creación de tablas en la base de datos

In [125]:
create_table_queries = [
    dropear_tablas,
    crear_peliculas,
    crear_artistas,
    crear_peliculas_artistas
]

conn = dcs.connect_to_database("prueba_tecnica", credentials_dict=database_credentials, autocommit=True)

# dropear tablas y crear
for query in create_table_queries:
    conn.cursor().execute(query)

### 3.2.1 Inserción peliculas

Preparación tabla peliculas:

In [126]:
peliculas_db = pd.concat([peliculas, detalles_peliculas],axis=1)
peliculas_db.head(1)

,Tipo,Titulo,Año,Mes,Id,generos,titulo,cali_imdb,direc_imbd,guion_imbd,argum_imbd,durac_imbd,id_pelicula
0,Movie,Las Travesuras de Super Chido,1990.0,10.0,tt0320656,Action,Las Travesuras de Super Chido,7.7,Alejandro Todd,Óscar FentanesJuan Garrido,NaN,1h 35min,tt0320656


In [127]:
peliculas_db = peliculas_db[["Id", "Titulo","Tipo","generos", "Año", "Mes","cali_imdb",
              "direc_imbd","guion_imbd", "argum_imbd","durac_imbd"]]

In [128]:
conn.cursor().executemany(
"INSERT INTO peliculas VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
list(peliculas_db.itertuples(index=False, name=None))
)

### 3.2.2 Inserción artistas

Preparación tabla artistas:

In [129]:
artistas_db = detalles_artistas[["actr", "año_nac", "que_hace", "premios"]]
artistas_db.head(1)

,actr,año_nac,que_hace,premios
0,A. Gunaseelan,NaN,"Casting Department, Actress, Casting Director",1 nomination


In [130]:
conn.cursor().executemany(
"INSERT INTO artistas VALUES (%s,%s,%s,%s)",
list(artistas_db.itertuples(index=False, name=None))
)

### 3.2.3 Inserción peliculas artistas

Preparación tabla artistas:

In [131]:
pelicula_artistas_db = (pelicula_artista.dropna(subset="Actriz_Actor")
                .merge(artistas_db["actr"], left_on="Actriz_Actor",right_on="actr",how="inner")[["Movie ID","Actriz_Actor"]])

In [132]:
conn.cursor().executemany(
"INSERT INTO peliculas_artistas(id_pelicula,actriz_actor) VALUES (%s,%s)",
list(pelicula_artistas_db.itertuples(index=False, name=None))
)